In [1]:
import pickle
import numpy as np
import codecs
import random

from nltk.tokenize import wordpunct_tokenize

In [2]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_TRAIN_DATASET.pickle", "rb") as f:
    train_dataset = pickle.load(f)

In [3]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_VALID_DATASET.pickle", "rb") as f:
    valid_dataset = pickle.load(f)

In [4]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_TEST_DATASET.pickle", "rb") as f:
    test_dataset = pickle.load(f)

In [5]:
len(train_dataset)

1068

In [6]:
len(valid_dataset)

134

In [7]:
len(test_dataset)

134

------------------------------------------------------------------------------------------------------

## Produce same output as the script *create_dataset* from data2text-plain

### Plan

section_content -------- true text   

entity_recognition  ------- actually input - set of records   

Set of records in order they appear in section_content    -------- content plan 

In [8]:
RECORD_DELIM = " "
DELIM = u"￨"

HOME = "HOME"
AWAY = "AWAY"

# ENTITY = "Indication"
ENTITY = "Section4"

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
UNK = 0
BOS_WORD = '<s>'
EOS_WORD = '</s>'

In [9]:
# make sure - sorted entities
def _sort_key(entity):
    return entity['BeginOffset']

def test_order_entities(section_entities):
    
    sorted_entities = sorted(section_entities, key=_sort_key)
    
    if sorted_entities == section_entities: return True
    
    return False


def test_order_dataset(dataset):
    for leaflet in dataset:

        current_leaflet_sections = [leaflet.section1, leaflet.section2, 
                                    leaflet.section3, leaflet.section4, 
                                    leaflet.section5, leaflet.section6]

        for current_section in current_leaflet_sections:

            if current_section.entity_recognition is None:
                continue

            assert test_order_entities(current_section.entity_recognition) == True

            
test_order_dataset(train_dataset)
test_order_dataset(valid_dataset)
test_order_dataset(test_dataset)

In [10]:
def create_summary_contentplan(dataset):
    """
    Transform dataset to be a suitable format for model data2text-plan
    """
    
    # array to store section1 of each leaflet
    summaries_leaflets = []
    
    # array to store content plan of each leaflet
    content_plan_leaflets = []
    
    for leaflet in dataset:
        
        # extract section1 content
        section1_content = leaflet.section4.section_content
        # extract results of NER
        section1_entity_recognition = leaflet.section4.entity_recognition
        
        # skip if either input or output is None
        if section1_content is None or section1_entity_recognition is None:
            continue
        
        # get the content plan of each section
        content_plan_section1 = ''

        for entity in section1_entity_recognition:
            entity_value = entity['Text'] if len(entity['Text'].split(" ")) == 0 else ("_").join(entity['Text'].split(" "))
            entity_type = entity['Type'] if entity['Type'] is not None and len(entity['Type']) > 0 else entity['Category']
            
            # randomly choose HOME or AWAY
            if random.randint(1,2) == 1:
                content_plan_section1 += entity_value + DELIM + ENTITY + DELIM + entity_type + DELIM + HOME
            else:
                content_plan_section1 += entity_value + DELIM + ENTITY + DELIM + entity_type + DELIM + AWAY
            
            if section1_entity_recognition.index(entity) != len(section1_entity_recognition) - 1:
                content_plan_section1 += " "
            else:
                content_plan_section1 += " " + "\n"


        content_plan_leaflets.append(content_plan_section1)

        # get the section1 content
        # make sure to have punctuations as a separate token
        section1_content = wordpunct_tokenize(section1_content)
        
        # back to string
        section1_content = " ".join(section1_content)
        
        # add "\n" at the end
        section1_content = section1_content + "\n"
        
        summaries_leaflets.append(section1_content)
    
    return (content_plan_leaflets, summaries_leaflets)

In [11]:
def add_special_records(records):
    """
    To src_train.txt and src_valid.txt pre-append these special characters, according to data2text-plan project
    """
    
    record = []
    record.append(UNK_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(BOS_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(EOS_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    
    return records

In [12]:
# get the src_train - training data to be input to the model

def create_src_table(content_plan_leaflets):
    """
    Create src_train - input "table" to the model
    
    Idea: - we do not have a table, so randomized the records in content plan
    
    Update: - do not randomize - make it easier for the model to learn
    """
    
    # store input "table" of each leaflet in array
    src_leaflets = []
    
    for leaflet_content_plan in content_plan_leaflets:
        # remove the end symbol ('\n') of the string
        leaflet_content_plan = leaflet_content_plan[:-2]

        # split string into a list of records
        leaflet_content_plan_collection = leaflet_content_plan.split(" ")
        
        # do not do
        # randomly shuffle records in a list
        # random.shuffle(leaflet_content_plan_collection)
        
        # add special symbols to the begining
        special_symbols = []
        special_symbols = add_special_records(special_symbols)
        
        # create a string containing all the records and special_symbols in the begining
        src_leaflet_section1 = ''

        src_leaflet_section1 += " ".join(special_symbols)
        src_leaflet_section1 += " "

        src_leaflet_section1 += " ".join(leaflet_content_plan_collection)
        src_leaflet_section1 += '\n'


        src_leaflets.append(src_leaflet_section1)
    
    return src_leaflets

In [13]:
def create_training_sets(dataset):
    
    # Output files
    INTER_CONTENT_PLAN = 'inter/train_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE = 'src_train.txt'  # src file input to first stage
    TRAIN_TGT_FILE = "tgt_train.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT = 'train_content_plan.txt'  # content plan output of first stage
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to corresponding files
    output_file = open(INTER_CONTENT_PLAN, 'w')
    for content_plan in content_plan_leaflets:
        output_file.write(content_plan)
    output_file.close()
    
    summary_file = open(TRAIN_TGT_FILE, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()
    
    ### create last file needed - e.g (rotowire/train_content_plan.txt)
    inputs = []
    content_plans = []
    with codecs.open(INTER_CONTENT_PLAN, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            content_plans.append(line.split())

    with codecs.open(SRC_FILE, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            inputs.append(line.split())
    
    # basically - now content plan POINTs to index in the training dataset
    # content_plan - collection of indexes where each index points to record in training dataset - training_dataset[index]
    
    outputs = []

    for i, training_sample in enumerate(inputs):
        content_plan = content_plans[i]
        output = []
        for record in content_plan:
            output.append(str(training_sample.index(record)))
        outputs.append(" ".join(output))
        
    # write to a file

    output_file = open(CONTENT_PLAN_OUT, 'w')

    # add \n to the end of the string
    output_file.write("\n".join(outputs))
    # add \n between content plans
    output_file.write("\n")

    output_file.close()
    
    return src_leaflets, content_plan_leaflets, summaries_leaflets

In [14]:
leaflets_src_train, leaflets_inter_contentplan_train, leaflets_tgt_train = create_training_sets(train_dataset)

In [15]:
with open('src_train.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    src_train = reader.readlines()

In [16]:
with open('tgt_train.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    tgt_train = reader.readlines()

In [17]:
with open('train_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    train_content_plan = reader.readlines()

In [18]:
with open('inter/train_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    inter_train_content_plan = reader.readlines()

======================================================================================================================
## Validation dataset 

In [26]:
def create_validation_sets(dataset):
    
    # Output files    
    INTER_CONTENT_PLAN_VALID = 'inter/valid_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE_VALID = 'src_valid.txt'  # src file input to first stage
    TRAIN_TGT_FILE_VALID = "tgt_valid.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT_VALID = 'valid_content_plan.txt'  # content plan output of first stage
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to corresponding files
    output_file = open(INTER_CONTENT_PLAN_VALID, 'w')
    for content_plan in content_plan_leaflets:
        output_file.write(content_plan)
    output_file.close()
    
    summary_file = open(TRAIN_TGT_FILE_VALID, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE_VALID, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()
    
    ### create last file needed - e.g (rotowire/train_content_plan.txt)
    inputs = []
    content_plans = []
    with codecs.open(INTER_CONTENT_PLAN_VALID, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            content_plans.append(line.split())

    with codecs.open(SRC_FILE_VALID, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            inputs.append(line.split())
    
    # basically - now content plan POINTs to index in the training dataset
    # content_plan - collection of indexes where each index points to record in training dataset - training_dataset[index]
    
    outputs = []

    for i, training_sample in enumerate(inputs):
        content_plan = content_plans[i]
        output = []
        for record in content_plan:
            output.append(str(training_sample.index(record)))
        outputs.append(" ".join(output))
        
    # write to a file

    output_file = open(CONTENT_PLAN_OUT_VALID, 'w')

    # add \n to the end of the string
    output_file.write("\n".join(outputs))
    # add \n between content plans
    output_file.write("\n")

    output_file.close()
    
    return src_leaflets, content_plan_leaflets, summaries_leaflets

In [27]:
leaflets_src_valid, leaflets_inter_contentplan_valid, leaflets_tgt_valid = create_validation_sets(valid_dataset)

In [37]:
with open('inter/valid_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    contentplan_valid = reader.readlines()

INTER_CONTENT_PLAN_VALID = 'inter/valid_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE_VALID = 'src_valid.txt'  # src file input to first stage
    TRAIN_TGT_FILE_VALID = "tgt_valid.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT_VALID = 'valid_content_plan.txt'  # content plan output of first stage

================================================================================================================================
## Test dataset 

In [28]:
def create_test_sets(dataset):
    
    # Output files
    SRC_FILE_TEST = 'test/src_test.txt'  # src file input to first stage
    TRAIN_TGT_FILE_TEST = "test/tgt_test.txt"  # tgt file of second stage 
    
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to just summary and src data
    
    summary_file = open(TRAIN_TGT_FILE_TEST, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE_TEST, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()

    return src_leaflets, summaries_leaflets

In [29]:
leaflets_src_test, leaflets_tgt_test = create_test_sets(test_dataset)

=======================================================================================================================
## Creating *train-roto-ptrs.txt*

In [30]:
# Output files
INTER_CONTENT_PLAN = 'inter/train_content_plan.txt'  # intermediate content plan input to second stage
TRAIN_TGT_FILE = "tgt_train.txt"  # tgt file of second stage
OUTPUT_FILE = "train-roto-ptrs.txt"

In [31]:
with open(TRAIN_TGT_FILE) as reader:
    leaflet_tgt_train = reader.readlines()

In [32]:
with open(INTER_CONTENT_PLAN) as reader:
    leaflets_inter_content_plan = reader.readlines()

For eg: the last entry 245,39 in train_roto_ptrs[1] indicates that the 245th token in summary matches with 39th content plan entry.  

Phoenix ----> Phoenix￨Suns￨TEAM-CITY￨HOME  
Suns ----> Suns￨Suns￨TEAM-NAME￨HOME  
39 ----> 39￨Suns￨TEAM-WINS￨HOME  
38 ----> 38￨Suns￨TEAM-LOSSES￨HOME  
87 ----> 87￨Suns￨TEAM-PTS￨HOME  
85 ----> 85￨Jazz￨TEAM-PTS￨AWAY  
Utah ----> Utah￨Jazz￨TEAM-CITY￨AWAY  

In [33]:
leaflet_tgt_train[0]

"like all medicines , cystagon can cause side effects , although not everybody gets them . cystagon may cause some people to become drowsy or less vigilant than they are normally . make sure you know how you or your child reacts to this medicine before doing anything that could be dangerous if not alert . the following side effects were reported as follows : very common ( occurring in at least one in 10 patients ), common ( occurring in at least one in 100 patients ), uncommon ( occurring in at least one in1 , 000 patients ), rare ( occurring in at least one in 10 , 000 patients ), very rare ( occurring in at least one in 100 , 000 patients ). - very common : vomiting , nausea , diarrhoea , loss of appetite , fever and sensation of sleep - common : abdominal pain or discomfort , unpleasant breath and body odour , skin eruption , gastroenteritis , fatigue , headache , encephalopathy ( brain disorder ) and liver function test abnormalities . - uncommon : skin striae , skin lesion ( littl

In [34]:
leaflets_inter_content_plan[0]

'cystagon￨Section4￨PRODUCT_NAME￨AWAY all_medicines￨Section4￨TREATMENT￨HOME cystagon￨Section4￨GENERIC_NAME￨HOME side_effects￨Section4￨DX_NAME￨AWAY cystagon￨Section4￨GENERIC_NAME￨AWAY drowsy￨Section4￨DX_NAME￨HOME alert￨Section4￨DX_NAME￨HOME side_effects￨Section4￨DX_NAME￨HOME 10￨Section4￨NUMBER￨HOME 100￨Section4￨NUMBER￨HOME 10,000￨Section4￨NUMBER￨AWAY 100,000￨Section4￨NUMBER￨AWAY vomiting￨Section4￨DX_NAME￨AWAY nausea￨Section4￨DX_NAME￨AWAY diarrhoea￨Section4￨DX_NAME￨AWAY loss_of_appetite￨Section4￨DX_NAME￨AWAY fever￨Section4￨DX_NAME￨HOME sensation_of_sleep￨Section4￨DX_NAME￨AWAY abdominal_pain￨Section4￨DX_NAME￨AWAY discomfort￨Section4￨DX_NAME￨HOME unpleasant_breath￨Section4￨DX_NAME￨HOME body_odour￨Section4￨DX_NAME￨AWAY skin_eruption￨Section4￨DX_NAME￨AWAY gastroenteritis￨Section4￨DX_NAME￨HOME fatigue￨Section4￨DX_NAME￨AWAY headache￨Section4￨DX_NAME￨AWAY encephalopathy_(brain_disorder￨Section4￨PROBLEM￨AWAY liver_function_test_abnormalities￨Section4￨DX_NAME￨AWAY skin_striae￨Section4￨DX_NAME￨HOME

In [35]:
roto_pts_content = []

# for each leaflet
for leflet_num in range(len(leaflet_tgt_train)):
    
    # get current leaflet and content plan
    current_leaflet = leaflet_tgt_train[leflet_num].split()
    current_content_plan = leaflets_inter_content_plan[leflet_num].split()
    
    # get the values of content plan
    instances = []
    for entry in current_content_plan:
        record_values = entry.split(DELIM)[0]
        instances.append(record_values)
    
    # pairs (index_tgt, index_contentplan) for each leaflet
    current_str = []
    
    # for each token in current summary
    for token_pos in range(len(current_leaflet)):
        
        # get token
        token = current_leaflet[token_pos]
        
        # possible tokens if 2 words in content plan like 'immunodeficiency_syndrome'
        if token_pos < (len(current_leaflet)-1):
            token_2words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1]
        else:
            token_2words = 'something that would never be in the section content'
        
        ### my-new-change
        # possible tokens if 3 words in content plan
        if token_pos < (len(current_leaflet)-2):
            token_3words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1] + "_" + current_leaflet[token_pos+2]
        else:
            token_3words = 'something that would never be in the section content'
        
        # possible tokens if 4 words in content plan
        if token_pos < (len(current_leaflet)-3):
            token_4words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1] + "_" + current_leaflet[token_pos+2] + "_" + current_leaflet[token_pos+3]
        else:
            token_4words = 'something that would never be in the section content'
        
        
        for content_plan_index in range(len(instances)):
                
            if token_4words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)

                # find just one match
                break
            
            if token_3words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
            
            if token_2words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
            
            if token == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
    
    # join pairs into string with " " between pairs
    current_str = " ".join(current_str)
    
    # add "\n" at the end
    current_str += "\n"
    
    roto_pts_content.append(current_str)

In [36]:
OUTPUT_FILE = "train-roto-ptrs.txt"

src_file = open(OUTPUT_FILE, 'w')
for src_instance in roto_pts_content:
    src_file.write(src_instance)
src_file.close()

In [37]:
len(roto_pts_content)

1028

In [38]:
index = 500

In [39]:
roto_pts_content[index]

'1,1 4,2 8,3 19,4 24,5 27,6 28,31 33,7 36,8 38,9 41,10 42,11 53,0 58,13 63,14 65,15 67,16 72,17 81,18 84,19 92,20 94,21 99,22 103,23 107,24 109,25 112,26 115,27 118,54 123,28 133,29 147,32 152,92 166,93 172,33 175,34 177,35 180,36 183,38 185,37 191,88 198,12 205,40 207,41 209,42 214,44 228,47 230,48 240,49 242,50 249,51 273,39 281,55 285,56 287,57 289,58 293,59 298,60 301,61 308,62 311,63 313,64 316,65 318,66 320,67 322,68 326,69 329,70 330,90 333,71 336,72 338,73 367,77 383,80 387,81 396,82 401,84 408,85 428,89 437,91 443,94 449,95 463,97 485,96 502,98\n'

In [40]:
len(roto_pts_content[index].split())

85

In [41]:
len(leaflets_inter_content_plan[index].split())

99

### Check whether correct

In [42]:
content_plan_indeces = []

for pair in roto_pts_content[index].split():
    pair = pair.split(",")
    
    a = int(pair[0])
    b = int(pair[1])
    
    content_plan_indeces.append(b)
    
    print(leaflet_tgt_train[index].split()[a:a+4], end=" ----> ")
    print(leaflets_inter_content_plan[index].split()[b])
    

['all', 'medicines', ',', 'this'] ----> all_medicines￨Section4￨TREATMENT￨HOME
['this', 'medicine', 'can', 'cause'] ----> this_medicine￨Section4￨TREATMENT￨AWAY
['side', 'effects', ',', 'although'] ----> side_effects￨Section4￨DX_NAME￨HOME
['side', 'effects', 'may', 'happen'] ----> side_effects￨Section4￨DX_NAME￨AWAY
['this', 'medicine', ':', 'serious'] ----> this_medicine￨Section4￨TREATMENT￨HOME
['serious', 'side', 'effects', 'uncommon'] ----> serious_side_effects￨Section4￨PROBLEM￨AWAY
['side', 'effects', 'uncommon', '('] ----> side_effects￨Section4￨DX_NAME￨HOME
['affect', 'up', 'to', '1'] ----> affect￨Section4￨DX_NAME￨AWAY
['1', 'in', '100', 'people'] ----> 1￨Section4￨NUMBER￨AWAY
['100', 'people', ')', 'immediate'] ----> 100￨Section4￨NUMBER￨AWAY
['immediate', 'breathing', 'difficulties', ':'] ----> immediate_breathing_difficulties￨Section4￨PROBLEM￨AWAY
['breathing', 'difficulties', ':', 'if'] ----> breathing_difficulties￨Section4￨DX_NAME￨AWAY
['trixeo', 'aerosphere', ',', 'such'] ----> t

In [43]:
# check out pairs missed
for i in range(0, len(leaflets_inter_content_plan[index].split()), 1):
    if i not in content_plan_indeces:
        print(leaflets_inter_content_plan[index].split()[i])
        
# explanation - 3-word long token
# explanation - hiv_infection ---> bc NER outputs - 'hiv', 'hiv_infection' - in content plan I have 2 tokens starting with 'hiv'

the_serious_side_effects_above￨Section4￨PROBLEM￨HOME
nausea)￨Section4￨PROBLEM￨HOME
a_hoarse_voice_muscle_cramps￨Section4￨PROBLEM￨AWAY
awareness_of_your_heart_beating￨Section4￨DX_NAME￨AWAY
pneumonia_(infection_of_the_lung￨Section4￨PROBLEM￨AWAY
trixeo_aerosphere￨Section4￨TREATMENT￨AWAY
agitated_depression_fast_heart_beat￨Section4￨PROBLEM￨AWAY
uneven_heart_beat_chest_pain￨Section4￨PROBLEM￨AWAY
tightening_in_the_chest_(angina_pectoris)￨Section4￨PROBLEM￨AWAY
10,000￨Section4￨NUMBER￨AWAY
changes_in_behaviour_an_effect￨Section4￨DX_NAME￨AWAY
clouding_of_the_lens_of_your_eyes￨Section4￨DX_NAME￨AWAY
increased_pressure_in_the_eye￨Section4￨PROBLEM￨AWAY
glaucoma)_swelling_of_your_face￨Section4￨PROBLEM￨AWAY


In [44]:
leaflet_tgt_train[500]

'like all medicines , this medicine can cause side effects , although not everybody gets them . the following side effects may happen with this medicine : serious side effects uncommon ( may affect up to 1 in 100 people ) immediate breathing difficulties : if you get breathing difficulties straight after using trixeo aerosphere , such as tightness of the chest , coughing , wheezing or feeling breathless , stop using this medicine and tell your doctor straight away . allergic reactions : swelling of your face , particularly around your mouth ( swelling of your tongue or throat may make it difficult to swallow ) rash or hives together with difficulty breathing suddenly feeling faint these symptoms may be signs of an allergic reaction which may become serious . stop using this medicine and call for medical help straight away if you notice the serious side effects above . other side effects tell your doctor or pharmacist if you notice any of the following side effects : common ( may affect

In [45]:
leaflets_inter_content_plan[500]

'trixeo_aerosphere￨Section4￨PRODUCT_NAME￨AWAY all_medicines￨Section4￨TREATMENT￨HOME this_medicine￨Section4￨TREATMENT￨AWAY side_effects￨Section4￨DX_NAME￨HOME side_effects￨Section4￨DX_NAME￨AWAY this_medicine￨Section4￨TREATMENT￨HOME serious_side_effects￨Section4￨PROBLEM￨AWAY affect￨Section4￨DX_NAME￨AWAY 1￨Section4￨NUMBER￨AWAY 100￨Section4￨NUMBER￨AWAY immediate_breathing_difficulties￨Section4￨PROBLEM￨AWAY breathing_difficulties￨Section4￨DX_NAME￨AWAY trixeo_aerosphere￨Section4￨TREATMENT￨AWAY tightness_of_the_chest￨Section4￨PROBLEM￨AWAY coughing￨Section4￨DX_NAME￨AWAY wheezing￨Section4￨DX_NAME￨HOME feeling_breathless￨Section4￨DX_NAME￨HOME this_medicine￨Section4￨TREATMENT￨AWAY allergic_reactions￨Section4￨DX_NAME￨HOME swelling_of_your_face￨Section4￨PROBLEM￨AWAY mouth￨Section4￨SYSTEM_ORGAN_SITE￨AWAY swelling_of_your_tongue￨Section4￨PROBLEM￨HOME throat￨Section4￨SYSTEM_ORGAN_SITE￨HOME difficult_to_swallow￨Section4￨DX_NAME￨HOME rash￨Section4￨DX_NAME￨AWAY hives￨Section4￨DX_NAME￨AWAY difficulty_breat